In [19]:
import numpy as np
import scipy.optimize as sci_plt
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

from sklearn.preprocessing import StandardScaler
from pprint import pprint

pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)



def get_data(symbol):
    # Define the endpoint URL
    url = f"https://api.coingecko.com/api/v3/coins/{symbol}/ohlc?vs_currency=usd&days=365"

    # Send a GET request to the CoinGecko API
    response = requests.get(url)

    # Raise an exception if the request was unsuccessful
    if not response.ok:
        response.raise_for_status()

    # Convert the response data to a pandas DataFrame
    df = pd.DataFrame(response.json(), columns=['time', 'open', 'high', 'low', 'close'])

    # Convert the time column to datetime format
    df['time'] = pd.to_datetime(df['time'], unit='ms')

    # Save the DataFrame as a CSV file
    df.to_csv(f"{symbol}_ohlc.csv", index=False)

    return df

# Fetch and save data for each symbol
#symbols = ['canto', 'internet-computer', 'evmos', 'immutable-x', 'ocean-protocol', 'solana', 'avalanche', 'ethereum']

# for symbol in symbols:
#     get_data(symbol)
#     time.sleep(1)  # Delay for 10 seconds

symbols = ['canto', 'internet-computer', 'evmos', 'immutable-x', 'ocean-protocol', 'solana']

# Initialize an empty DataFrame to hold all the close prices
close_prices = pd.DataFrame()

for symbol in symbols:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(f"{symbol}_ohlc.csv")

    # Select the 'time' and 'close' columns and rename the 'close' column to the symbol name
    df = df[['time', 'close']].rename(columns={'close': symbol})

    # Set the 'time' column as the index
    df.set_index('time', inplace=True)

    # Join the DataFrame to the close_prices DataFrame
    if close_prices.empty:
        close_prices = df
    else:
        close_prices = close_prices.join(df, how='outer')

# Print the close_prices DataFrame
df = close_prices.copy()
df = df.dropna()

df


,canto,internet-computer,evmos,immutable-x,ocean-protocol,solana
time,,,,,,
2023-01-03,0.078749,3.97,0.304506,0.399452,0.167662,11.27
2023-01-07,0.104647,3.91,0.451186,0.422230,0.190703,13.52
2023-01-11,0.107169,4.31,0.413388,0.495145,0.233204,16.20
2023-01-15,0.132911,5.12,0.505522,0.560912,0.263717,24.30
2023-01-19,0.115681,4.96,0.427515,0.579389,0.274099,20.86
...,...,...,...,...,...,...
2023-12-19,0.281052,9.78,0.167222,2.170000,0.507857,74.34
2023-12-23,0.272059,9.30,0.156616,2.300000,0.543098,98.08
2023-12-27,0.315718,9.25,0.138640,2.360000,0.561256,112.33


In [20]:
# Define a dictionary that maps the old column names to the new ones
column_names = {
    'canto': 'CANTO',
    'internet-computer': 'ICP',
    'evmos': 'EVMOS',
    'immutable-x': 'IMX',
    'ocean-protocol': 'OCEAN',
    'solana': 'SOL'
}

# Rename the columns
df.rename(columns=column_names, inplace=True)

df

,CANTO,ICP,EVMOS,IMX,OCEAN,SOL
time,,,,,,
2023-01-03,0.078749,3.97,0.304506,0.399452,0.167662,11.27
2023-01-07,0.104647,3.91,0.451186,0.422230,0.190703,13.52
2023-01-11,0.107169,4.31,0.413388,0.495145,0.233204,16.20
2023-01-15,0.132911,5.12,0.505522,0.560912,0.263717,24.30
2023-01-19,0.115681,4.96,0.427515,0.579389,0.274099,20.86
...,...,...,...,...,...,...
2023-12-19,0.281052,9.78,0.167222,2.170000,0.507857,74.34
2023-12-23,0.272059,9.30,0.156616,2.300000,0.543098,98.08
2023-12-27,0.315718,9.25,0.138640,2.360000,0.561256,112.33


In [21]:
log_return = np.log(1 + df.pct_change())

random_weights = np.array(np.random.random(6))
rebalance_weights = random_weights / np.sum(random_weights)
display(rebalance_weights)

# Expected return
exp_ret = np.sum((log_return.mean() * rebalance_weights) * 365)

# Expected volatility
exp_vol = np.sqrt(np.dot(rebalance_weights.T, np.dot(log_return.cov() * 365, rebalance_weights)))

# Sharpe Ratio
SR = exp_ret / exp_vol

# Put the weights in a DataFrame to view them
data = {'random_weights': random_weights, 'rebalance_weights': rebalance_weights}
weights_df = pd.DataFrame(data=data, index=df.columns)

print(' ')
print('='*80)
print('-'*80)
print(weights_df)
print('-'*80)

#do the same with the other metrics

metrics_df = pd.DataFrame(data={'Expected Portfolio Return': exp_ret, 'Expected Portfolio Volatility': exp_vol, 'Sharpe Ratio': SR}, index=[0])

print('-'*80)
print('Portfolio Metrics')
print(metrics_df)
print('-'*80)


array([0.04263061, 0.06400891, 0.31052828, 0.21792981, 0.06501962,
       0.29988277])

 
--------------------------------------------------------------------------------
       random_weights  rebalance_weights
CANTO        0.111929           0.042631
ICP          0.168058           0.064009
EVMOS        0.815305           0.310528
IMX          0.572184           0.217930
OCEAN        0.170712           0.065020
SOL          0.787355           0.299883
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Portfolio Metrics
   Expected Portfolio Return  Expected Portfolio Volatility  Sharpe Ratio
0                   3.698753                        1.50815      2.452511
--------------------------------------------------------------------------------


# Portfolio Optimization: Monte Carlo

In [22]:
num_of_portfolios = 10000

#weights array
all_weights = np.zeros((num_of_portfolios, len(df.columns)))

ret_arr = np.zeros(num_of_portfolios) #expected return
vol_arr = np.zeros(num_of_portfolios) #expected volatility
sharpe_arr = np.zeros(num_of_portfolios) #sharpe ratio

for n in range(num_of_portfolios):
    #weights
    weights = np.array(np.random.random(6))
    weights = weights / np.sum(weights)

    #save weights
    all_weights[n,:] = weights

    #expected return
    ret_arr[n] = np.sum((log_return.mean() * weights) * 365)

    #expected volatility
    vol_arr[n] = np.sqrt(np.dot(weights.T, np.dot(log_return.cov() * 365, weights)))

    #sharpe ratio
    sharpe_arr[n] = ret_arr[n] / vol_arr[n]
    

In [23]:
#combine the arrays into a dataframe
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]

simulations_df = pd.DataFrame(data=simulations_data).T

simulations_df.columns = ['Expected Portfolio Return', 'Expected Portfolio Volatility', 'Sharpe Ratio', 'Portfolio Weights']

simulations_df = simulations_df.infer_objects()

print(' ')
print('-'*80)
print('Simulations Dataframe')
print('-'*80)
print(simulations_df.head())
print('-'*80)

 
--------------------------------------------------------------------------------
Simulations Dataframe
--------------------------------------------------------------------------------
   Expected Portfolio Return  Expected Portfolio Volatility  Sharpe Ratio                                                                                                               Portfolio Weights
0                   3.909899                       1.488995      2.625865  [0.16218821913291975, 0.04449060161864236, 0.24920463882491106, 0.04093505582239065, 0.20624759026340606, 0.29693389433773015]
1                   5.243521                       1.400744      3.743382   [0.10664696926691454, 0.22417968257668908, 0.10170940926484844, 0.14688741194983784, 0.12857622516477205, 0.2920003017769381]
2                   3.306307                       1.445473      2.287353   [0.1622370680646489, 0.14382178136364784, 0.2454596870518865, 0.18920712800777248, 0.19992076896488137, 0.059353566547162874]
3     

In [27]:
#grab the index of the max sharpe ratio
max_sharpe_index = simulations_df['Sharpe Ratio'].idxmax()
simulations_df.iloc[max_sharpe_index]

Expected Portfolio Return                                                                                                                              7.020621
Expected Portfolio Volatility                                                                                                                           1.56401
Sharpe Ratio                                                                                                                                            4.48886
Portfolio Weights                [0.042861413795017775, 0.08525471587351374, 0.005637164453239435, 0.3129318228344662, 0.06550752958975246, 0.4878073534540103]
Name: 3084, dtype: object

In [32]:
#extract the weights of the max sharpe ratio and match them with the symbols
max_sharpe_weights = simulations_df.iloc[max_sharpe_index]['Portfolio Weights']
max_sharpe_weights_df = pd.DataFrame(data=max_sharpe_weights, index=df.columns, columns=['Max Sharpe Ratio Weights'])
#round the columns to 2 decimal places
max_sharpe_weights_df = max_sharpe_weights_df.round(3)
#show in percentage
max_sharpe_weights_df = max_sharpe_weights_df * 100
print(' ')
print('-'*80)
print('Max Sharpe Ratio Weights')
print('-'*80)
print(max_sharpe_weights_df)
print('-'*80)

 
--------------------------------------------------------------------------------
Max Sharpe Ratio Weights
--------------------------------------------------------------------------------
       Max Sharpe Ratio Weights
CANTO                       4.3
ICP                         8.5
EVMOS                       0.6
IMX                        31.3
OCEAN                       6.6
SOL                        48.8
--------------------------------------------------------------------------------
